In [1]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping
import RNA
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle

/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Chargement du modèle et prédictions : 

In [ ]:
# paramètres 

matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)

c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)

c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Dropout(0.2)(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)

interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

interaction.load_weights('model_rna-rna_deep_livrable.h5')

#### Construction des inputs et fenêtrage :

bdd : base de dimensions (l,2) à tester les deux colonnes correspondent respectivement aux deux séquences primaires d'entrée, au format string (ex : 'acgugcua' et 'auugaucgau') 

Test de deep_rna_rna sur les données de bdd : fenêtrage  de vitesse s=12 sur les deux séquences :
- la fenêtre mesure 25 nucléotides (paramètre f)
- deep rna rna prédit pour chaue couple de fenêtres si ces deux sites sont suceptibles d'interagir ensemble

Le résultat est donc une matrice de taille (int((dim-f+1)/s),int((dim2-f+1)/s)).

In [ ]:
# f longueur des sites à tester sur chaque séquence
f = 25
# s vitesse de glissement de la fenêtre de test le long des deux séquences
s = 12
# facteur_seuil afin de décider quelles prédictions sont considérées comme des vraies interactions
facteur_seuil = 0.95

for r in range(len(bdd)):
    
    seq1 = bdd[r,0]
    seq2 = bdd[r,1]
    dim = len(seq1)
    dim2 = len(seq2)
    matrice = np.zeros((1,dim,dim2,1)) 
    matrice1 = np.zeros((1,dim,dim2,1))     
    matrice2 = np.zeros((1,dim,4))
    matrice3 = np.zeros((1,dim2,4))

    proba1 = np.array(RNA.pfl_fold_up(seq1,1,dim,dim))[:,1]
    proba2 = np.array(RNA.pfl_fold_up(seq2,1,dim2,dim2))[:,1]

    for j in range(dim):
        for k in range(dim2):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[0,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[0,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[0,j,k,0] = 1
            matrice1[0,j,k,0] = proba1[j] + proba2[k]
    for j in range(dim):
        if seq1[j]=='a':
            matrice2[0,j,0] = 1
        elif seq1[j]=='u':
            matrice2[0,j,1] = 1
        elif seq1[j]=='g':
            matrice2[0,j,2] = 1
        elif seq1[j]=='c':
            matrice2[0,j,3] = 1
    for j in range(dim2):
        if seq2[j]=='a':
            matrice3[0,j,0] = 1
        elif seq2[j]=='u':
            matrice3[0,j,1] = 1
        elif seq2[j]=='g':
            matrice3[0,j,2] = 1
        elif seq2[j]=='c':
            matrice3[0,j,3] = 1
    u = int((dim-f+1)/s)
    u2 = int((dim2-f+1)/s)
    resultats = np.zeros((u,u2))

    for i in range(u):
        for j in range(u2):        
            test = []
            test.append(remplissage(matrice[:,i*s:i*s+36,j:j+36,:],f))
            test.append(remplissage(matrice1[:,i*s:i*s+36,j:j+36,:],f))            
            test.append(remplissage_seq(matrice2[:,i*s:i*s+36],f))
            rev = np.zeros((1,36,4))
            for q in range(36):
                rev[:,36-q-1,:] = matrice2[:,q,:]
            test.append(remplissage_seq(rev,f))
            test.append(remplissage_seq(matrice3[:,j*s:j*s+36],f))
            rev2 = np.zeros((1,36,4))
            for q in range(36):
                rev2[:,36-q-1,:] = matrice3[:,q,:]
            test.append(remplissage_seq(rev2,f))

            resultats[i,j] = interaction.predict(test)[:,1]

#### Explication d'une prédiction par méthode de gradient class activation maps : mise en valuer de motifs pertinents

In [ ]:
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys

#### Importation du modèle d'explication :

In [ ]:
# paramètres 

matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

from keras.layers import merge
from keras import Model
k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)
#cnn1d1 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d1_input)
c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)
#cnn1d2 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d2_input)
c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Dropout(0.2)(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

Fonctions utiles pour l'explication 

In [2]:
def target_category_loss_output_shape(input_shape):
    return input_shape

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def seq2augc(seq):
    string = ''
    for i in range(36):
        if seq[i][0]==1:
            string += 'a'
        elif seq[i][1]==1:
            string += 'u'
        elif seq[i][2]==1:
            string += 'g'
        elif seq[i][3]==1:
            string += 'c'
        else:
            a=0
    return string

def argumentsmax(argw):
    l=[]
    for r in argw:
        l.append(r)
    return l

- Ajout d'une couche récupérant la sortie du réseau de neurones afin d'apercevoir les activations de la classe 'interaction' 
- Ajustement du modèle et chargement des poids de Deep_rna_rna

In [ ]:
nb_classes = 2
category_index = 1
target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
interaction_output = Lambda(target_layer, output_shape = target_category_loss_output_shape)(interaction_output)
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])
interaction.load_weights("model_finaldeep.h5")

Couches dont on étudie la sortie : couches de convolution extrayant des motifs des séquences primaires

In [ ]:
interaction.layers[4].name,interaction.layers[5].name,interaction.layers[6].name,interaction.layers[7].name

Sélection des deux tronçons de séquence dont l'interaction doit être expliquée

In [ ]:
seq1 = 'aaauauagcuagucagcucguacgcaguacgucguagcgacuagcuagcuagcuaucgaucuagcauggcuacugaguca'
seq2 = 'auuagcucagcuacggggggggacgucgaucguacgcuagcugcuagcucgcuaaucgauuagcuagcuacguacagcucguacgc'
proba1 = [0.9, 0.5, ..., 0.8]
proba2 = [0.7, 0.12, ..., 0.2]
assert(len(seq1)==len(proba1))
assert(len(seq2)==len(proba2))

# pos1 : argument de la première séquence d'où débute le tronçon de 36 nucléotides à tester
pos1 = 49
# pos2 : argument de la deuxième séquence d'où débute le tronçon de 36 nucléotides à tester
pos2 = 12

l = 1
matrice = np.zeros((l,36,36,1)) 
matrice1 = np.zeros((l,36,36,1))
matrice2 = np.zeros((l,36,4))
matrice3 = np.zeros((l,36,4))
matrice4 = np.zeros((l,36,4))
matrice5 = np.zeros((l,36,4))

for i in range(l):
    # sélection du tronçon 1
    seq1 = seq1[pos1:pos1+36]
    prob1 = proba1[pos1:pos1+36]
    # sélection du tronçon 2
    seq2 = seq2[pos2:pos2+36]
    prob2 = proba2[pos2:pos2+36]
    
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[i,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[i,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[i,j,k,0] = 1
            matrice1[i,j,k,0] = prob1[j]+prob2[k]
    for j in range(len(seq1)):
        if seq1[j]=='a':
            matrice2[i,j,0] = 1
        elif seq1[j]=='u':
            matrice2[i,j,1] = 1
        elif seq1[j]=='g':
            matrice2[i,j,2] = 1
        elif seq1[j]=='c':
            matrice2[i,j,3] = 1
    for j in range(len(seq2)):
        if seq2[j]=='a':
            matrice3[i,j,0] = 1
        elif seq2[j]=='u':
            matrice3[i,j,1] = 1
        elif seq2[j]=='g':
            matrice3[i,j,2] = 1
        elif seq2[j]=='c':
            matrice3[i,j,3] = 1

for i in range(36):
    matrice4[:,36-i-1,:] = matrice2[:,i,:]
for i in range(36):
    matrice5[:,36-i-1,:] = matrice3[:,i,:]

testing = []
testing.append(matrice)
testing.append(matrice1)
testing.append(matrice2
testing.append(matrice4)
testing.append(matrice3)
testing.append(matrice5)

#### Lancement de Gradient Class Activation Maps et affichage des motifs

In [ ]:
motif1 = []
    
z1=0
# couche 1 : séquence 1 à l'endroit
layer_name = 'conv1d_1'
# fonction de récupération de la sortie du réseau lié à la classe 'interaction'
loss = K.sum(interaction.layers[-1].output)
#récupération de la sortie de la couche de convolution
conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
# calcul des gradients
grads = normalize(K.gradients(loss, conv_output)[0])
gradient_function = K.function(interaction.input, [conv_output, grads])

output, grads_val = gradient_function(testing)
output, grads_val = output[0, :], grads_val[0, :, :]

# calcul du lien entre la séquence observée et les poids liant les filtres à la classe 'interaction'

weights = np.mean(grads_val, axis = 0)
cam = np.ones(output.shape[0 : 1], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w*output[:, i]
# cam = class activation map

cam = np.maximum(cam, 0)
# heatmap : normalisation de la cam

heatmap = cam / np.max(cam)

# Extraction des motifs repérés dans la séquence

args = argumentsmax(np.argwhere(heatmap==1)[:,0])
if (len(args)==0):
    z1+=1
else:    
    seq = seq2augc(matrice2[0])
    for pos in argumentsmax(args):
        motif1.append(seq[pos:pos+7])
# Idem pour la séquence 1 à l'envers
layer_name = 'conv1d_2'
loss = K.sum(interaction.layers[-1].output)
conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
grads = normalize(K.gradients(loss, conv_output)[0])
gradient_function = K.function(interaction.input, [conv_output, grads])

output, grads_val = gradient_function(testing)
output, grads_val = output[0, :], grads_val[0, :, :]

weights = np.mean(grads_val, axis = 0)
cam = np.ones(output.shape[0 : 1], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, i]

cam = np.maximum(cam, 0)
heatmap = cam / np.max(cam)
args = argumentsmax(np.argwhere(heatmap==1)[:,0])
if (len(args)==0):
    z1+=1
else:    
    seq = seq2augc(matrice4[0])
    for pos in argumentsmax(args):
        motif1.append(seq[pos:pos+7])

# Idem pour la séquence 2 à l'endroit

motif2 = []

z2=0
layer_name = 'conv1d_3'
loss = K.sum(interaction.layers[-1].output)
conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
grads = normalize(K.gradients(loss, conv_output)[0])
gradient_function = K.function(interaction.input, [conv_output, grads])

output, grads_val = gradient_function(testing)
output, grads_val = output[0, :], grads_val[0, :, :]

weights = np.mean(grads_val, axis = 0)
cam = np.ones(output.shape[0 : 1], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, i]

cam = np.maximum(cam, 0)
heatmap = cam / np.max(cam)

args = (np.argwhere(heatmap==1)[:,0])
if (len(args)==0):
    z2+=1
else:    
    seq = seq2augc(matrice3[0])
    for pos in argumentsmax(args):
        motif2.append(seq[pos:pos+7])

# Idem pour la séquence 2 à l'envers

layer_name = 'conv1d_4'
loss = K.sum(interaction.layers[-1].output)
conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
grads = normalize(K.gradients(loss, conv_output)[0])
gradient_function = K.function(interaction.input, [conv_output, grads])

output, grads_val = gradient_function(testing)
output, grads_val = output[0, :], grads_val[0, :, :]

weights = np.mean(grads_val, axis = 0)
cam = np.ones(output.shape[0 : 1], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, i]

cam = np.maximum(cam, 0)
heatmap = cam / np.max(cam)

args = (np.argwhere(heatmap==1)[:,0])
if (len(args)==0):
    z2+=1
else:    
    seq = seq2augc(matrice5[0])
    for pos in argumentsmax(args):
        motif2.append(seq[pos:pos+7])
#
if (z1>=2 or z2>=2):
    print('rien')
else:
    motifs.append([motif1,motif2])

print(motifs)

Les couples de motifs ainsi récupérés sur chaque séquence peuvent révéler des fonctions biologiques expliquant certaines interactions. 